<a href="https://colab.research.google.com/github/shreyansh273/Football-Scouting/blob/main/Player_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv

In [2]:
stats_name = {
#standard(stats)
'stats' : ["player","nationality","position","squad","age","birth_year", "games","games_starts","minutes",'minutes_90s',"goals","assists",'non_penalty_goals',"pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa",'npxg_xa',"xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"],
#shooting(shooting)
'shooting' : ["player","nationality","position","squad","age","birth_year","minutes_90s","goals","shots_total","shots_on_target","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","distance_from_goal","shots_free_kicks","pens_made","pens_att","xg","npxg","npxg_per_shot","xg_net","npxg_net"],
#passing(passing)
'passing' : ["player","nationality","position","squad","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"],
#goal and shot creation(gca)
'gca' : ["player","nationality","position","squad","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","sca_og_for","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_og_for"],
#defensive actions(defense)
'defense' : ["player","nationality","position","squad","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"],
#possession(possession)
'possession' : ["player","nationality","position","squad","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","pass_targets", "passes_received","passes_received_pct","progressive_passes_received"],
}

In [3]:
drop_columns = {
    'stats': ['index', 'Rk', 'Matches'],
    'shooting' : ['index', 'Rk', 'Matches'],
    'passing' : ['index', 'Rk', 'Matches'],
     'defense' : ['index', 'Rk', 'Matches','Tkl+Int'],
     'possession' : ['index', 'Rk', 'Matches'],
      'gca' : ['index', 'Rk', 'Matches']
}

In [12]:
leagues = [ [9,'Premier-League'], [13,'Ligue-1'], [20,'Bundesliga'], [11,'Serie-A'], [12,'La-Liga'] ]

In [19]:
def get_player_stats(league):
  stats_required = ['stats','shooting','passing','defense','possession','gca']
  combined_table = pd.DataFrame()
  for i in range(len(stats_required)):
    url = 'https://fbref.com/en/comps/'+str(league[0])+'/'+str(stats_required[i])+'/'+str(league[1])+'-Stats'
    html_content = requests.get(url).text.replace('<!--', '').replace('-->', '')
    df = pd.read_html(html_content)
    df[2].columns = df[2].columns.droplevel(0) # drop top header row
    df[2] = df[2][df[2]['Rk'].ne('Rk')].reset_index() # remove mid-table header rows 
    df[2].drop(drop_columns[stats_required[i]], axis=1, inplace=True)
    df[2].columns = stats_name[stats_required[i]]
    combined_table = pd.concat([combined_table, df[2]], axis=1)
    combined_table = combined_table.loc[:,~combined_table.columns.duplicated()]
  return combined_table

In [38]:
all_player_stats = pd.DataFrame(columns=['player'])
for i in range(len(leagues)):
  print(leagues[i][1])
  player_stats = get_player_stats(leagues[i])
  player_stats["League"] = leagues[i][1]
  all_player_stats = all_player_stats.append(player_stats)  

Premier-League
Ligue-1
Bundesliga
Serie-A
La-Liga


In [39]:
all_player_stats

,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,minutes_90s,goals,assists,non_penalty_goals,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,npxg_xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,shots_total,shots_on_target,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,distance_from_goal,shots_free_kicks,...,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,dribbles_completed_pct,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,progressive_carries,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,pass_targets,passes_received,passes_received_pct,progressive_passes_received,sca,sca_per90,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,sca_og_for,gca,gca_per90,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_og_for,League
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,29,1990,22,20,1777,19.7,0,1,0,0,0,2,0,0.00,0.05,0.05,0.00,0.05,1.2,1.2,0.8,2.0,0.06,0.04,0.10,0.06,0.10,18,6,33.3,0.91,0.30,0.00,0.00,20.3,3,...,72,405,643,344,38,1126,9,21,42.9,9,0,733,3522,1907,114,30,9,13,16,835,775,92.8,46,26,1.32,22,3,1,0,0,0,3,0.15,2,1,0,0,0,0,Premier-League
1,Tammy Abraham,eng ENG,FW,Chelsea,22,1997,22,12,1040,11.6,6,1,6,0,0,0,0,0.52,0.09,0.61,0.52,0.61,6.4,6.4,0.9,7.3,0.56,0.07,0.63,0.56,0.63,32,13,40.6,2.77,1.13,0.19,0.46,9.7,0,...,21,40,167,166,66,347,6,19,31.6,6,2,177,709,252,13,5,4,26,13,491,251,51.1,96,18,1.56,11,0,1,3,3,0,2,0.17,1,0,0,0,1,0,Premier-League
2,Che Adams,sct SCO,FW,Southampton,24,1996,36,30,2667,29.6,9,5,9,0,0,1,0,0.30,0.17,0.47,0.30,0.47,8.8,8.8,5.8,14.5,0.30,0.19,0.49,0.30,0.49,56,31,55.4,1.89,1.05,0.16,0.29,13.1,0,...,22,98,444,492,110,965,25,44,56.8,28,0,630,2627,1169,67,27,10,74,52,1474,734,49.8,189,66,2.23,48,0,1,6,8,3,8,0.27,7,0,0,0,1,0,Premier-League
3,Tosin Adarabioyo,eng ENG,DF,Fulham,22,1997,33,33,2953,32.8,0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,1.1,1.1,0.4,1.5,0.03,0.01,0.04,0.03,0.04,20,5,25.0,0.61,0.15,0.00,0.00,8.8,0,...,462,1338,920,73,34,2172,8,11,72.7,9,0,1324,6173,3455,50,6,0,5,8,1531,1491,97.4,5,17,0.52,16,0,0,1,0,0,1,0.03,1,0,0,0,0,0,Premier-League
4,Adrián,es ESP,GK,Liverpool,33,1987,3,3,270,3.0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0,0,NaN,0.00,0.00,NaN,NaN,NaN,0,...,102,113,0,0,0,84,0,0,NaN,0,0,50,162,74,0,0,0,0,0,47,46,97.9,0,0,0.00,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,Premier-League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,Joseba Zaldúa,es ESP,DF,Real Sociedad,28,1992,12,6,639,7.1,0,0,0,0,0,3,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.8,0.8,0.01,0.11,0.11,0.01,0.11,2,1,50.0,0.28,0.14,0.00,0.00,26.2,0,...,20,120,224,147,9,377,1,8,12.5,1,0,225,1040,567,32,13,1,8,2,276,257,93.1,17,10,1.41,9,1,0,0,0,0,0,0.00,0,0,0,0,0,0,La-Liga
578,Oier Zarraga,es ESP,MF,Athletic Club,21,1999,5,1,100,1.1,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.0,0.1,0.08,0.04,0.13,0.08,0.13,1,0,0.0,0.90,0.00,0.00,NaN,16.0,0,...,0,8,33,14,3,49,3,5,60.0,3,0,45,304,192,9,2,0,0,1,44,40,90.9,3,3,2.70,3,0,0,0,0,0,1,0.90,1,0,0,0,0,0,La-Liga
579,Igor Zubeldia,es ESP,DF,Real Sociedad,23,1997,24,21,1959,21.8,1,1,1,0,0,5,0,0.05,0.05,0.09,0.05,0.09,1.0,1.0,0.7,1.8,0.05,0.03,0.08,0.05,0.08,9,3,33.3,0.41,0.14,0.11,0.33,16.2,0,...,171,685,756,56,12,1343,7,9,77.8,7,0,927,6537,3715,72,4,0,7,4,937,922,98.4,4,10,0.46,10,0,0,0,0,0,2,0.09,2,0,0,0,0,0,La-Liga
580,Martín Zubimendi,es ESP,MF,Real Sociedad,21,1999,31,17,1882,20.9,0,0,0,0,0,6,0,0.00,0.00,0.00,0.00,0.00,0.6,0.6,0

In [40]:
all_player_stats[all_player_stats.player.str.contains("Ronaldo")]

,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,minutes_90s,goals,assists,non_penalty_goals,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,npxg_xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,shots_total,shots_on_target,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,distance_from_goal,shots_free_kicks,...,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,dribbles_completed_pct,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,progressive_carries,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,pass_targets,passes_received,passes_received_pct,progressive_passes_received,sca,sca_per90,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,sca_og_for,gca,gca_per90,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_og_for,League
494,Cristiano Ronaldo,pt POR,FW,Juventus,35,1985,33,31,2802,31.1,29,2,23,6,8,3,0,0.93,0.06,1.00,0.74,0.80,26.5,20.4,2.8,23.2,0.85,0.09,0.94,0.65,0.74,162,60,37.0,5.20,1.93,0.14,0.38,17.6,17,...,19,75,675,846,213,1398,58,90,64.4,59,2,1075,6441,4188,265,85,59,54,45,1678,1207,71.9,291,114,3.66,62,1,18,17,12,4,16,0.51,7,0,3,5,1,0,Serie-A
590,Ronaldo Vieira,gw GNB,MF,Hellas Verona,22,1998,4,2,226,2.5,0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.1,0.1,0.02,0.02,0.04,0.02,0.04,2,0,0.0,0.80,0.00,0.00,NaN,23.3,0,...,7,24,92,36,2,139,2,2,100.0,2,0,89,390,166,7,7,0,1,2,95,86,90.5,3,3,1.19,3,0,0,0,0,0,0,0.00,0,0,0,0,0,0,Serie-A


In [41]:
all_player_stats[all_player_stats.player.str.contains("Ødegaard")]

,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,minutes_90s,goals,assists,non_penalty_goals,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,npxg_xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,shots_total,shots_on_target,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,distance_from_goal,shots_free_kicks,...,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,dribbles_completed_pct,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,progressive_carries,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,pass_targets,passes_received,passes_received_pct,progressive_passes_received,sca,sca_per90,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,sca_og_for,gca,gca_per90,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_og_for,League
531,Martin Ødegaard,no NOR,MF,Arsenal,21,1998,14,9,866,9.6,1,2,1,0,0,0,0,0.10,0.21,0.31,0.10,0.31,1.1,1.1,2.1,3.2,0.11,0.22,0.33,0.11,0.33,15,3,20.0,1.56,0.31,0.07,0.33,20.5,1,...,6,68,275,289,26,540,12,18,66.7,12,0,457,2380,1183,78,17,5,12,11,547,458,83.7,38,42,4.36,34,4,1,1,2,0,6,0.62,5,0,0,0,1,0,Premier-League
581,Martin Ødegaard,no NOR,"MF,FW",Real Madrid,21,1998,7,3,234,2.6,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.4,0.5,0.04,0.15,0.19,0.04,0.19,3,0,0.0,1.15,0.00,0.00,NaN,19.9,0,...,0,27,103,76,6,180,1,1,100.0,1,0,134,768,366,20,8,1,2,4,177,159,89.8,21,6,2.31,5,1,0,0,0,0,0,0.00,0,0,0,0,0,0,La-Liga


In [42]:
all_player_stats.to_csv("all_players_stats.csv")

In [43]:
from google.colab import files
files.download("all_players_stats.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>